In [1]:
import numpy as np
import pandas as pd

In [3]:
import json
import re
from tqdm import tqdm_notebook as tqdm

In [5]:
import sys
sys.path.insert(1, '../tools/')
import os

In [6]:
from yelp_fusion import *

In [7]:
file_names = os.listdir('../data/raw_businesses_data/')

In [8]:
def is_json_file(in_str):
    businesses_info = re.findall(r'businesses_info.json$', in_str)
    reviews = re.findall(r'reviews.json$', in_str)
    if businesses_info:
        return 'businesses_info'
    elif reviews:
        return 'reviews'

In [9]:
file_businesses_info = [i for i in file_names if is_json_file(i) == 'businesses_info']
file_reviews = [i for i in file_names if is_json_file(i) == 'reviews']

# All Businesses

In [10]:
def load_json_to_df_businesses(file_name):
    with open('../data/raw_businesses_data/{}'.format(file_name), 'r') as file:
        data = json.load(file)

    return pd.DataFrame(data['businesses'])

In [11]:
all_businesses_info = [load_json_to_df_businesses(i) for i in tqdm(file_businesses_info)]

In [12]:
all_businesses_df = pd.concat(all_businesses_info, sort=False).reset_index(drop=True)

# All Reviews

In [16]:
def load_json_to_df_reviews(file_name):
    with open('../data/{}'.format(file_name), 'r') as file:
        data = json.load(file)

    return dict_to_df(data['reviews'])

In [17]:
all_reviews = [load_json_to_df_reviews(i) for i in tqdm(file_reviews)]

In [46]:
all_reviews_df = pd.concat(all_reviews, sort=False).reset_index(drop=True)

In [47]:
all_reviews_df.to_csv('../data/all_reviews_df.csv', index=False)

In [51]:
all_reviews_df.user[0]

{'id': 'Xri-uAd7HUGkO-uubNRR6Q',
 'profile_url': 'https://www.yelp.com/user_details?userid=Xri-uAd7HUGkO-uubNRR6Q',
 'image_url': 'https://s3-media3.fl.yelpcdn.com/photo/zfHhC21dwTcidSvMo2KDpw/o.jpg',
 'name': 'Kolby V.'}